# Setup

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from plotly.express import imshow as imshow_interactive

## Import app modules

In [ ]:
import sys
import pathlib

REPO_DIR = pathlib.Path(__name__).resolve().parent.parent
SRC_DIR = str(REPO_DIR / "src")
sys.path.append(SRC_DIR)

In [ ]:
from scanplot.io.read_image import read_image_rgb
from scanplot.io.saving import dump_coords_csv
from scanplot.plotting import draw_image

In [ ]:
from scanplot.core.template_match import template_match
from scanplot.core.preprocess import replace_black_pixels
from scanplot.core.corr_map_operations import normalize_map
from scanplot.core.process_template import (
    template_tresholding, reconstruct_template_mask
)
from scanplot.core.hough_transform import generalized_hough_transform
from scanplot.core.coords_converter import CoordinatesConverter

In [ ]:
from scanplot.view.detector_widget import DetectorWidget
from scanplot.view.coords_mapper_widget import CoordinatesMapper

# Algorithm Pipeline

## Upload plot image and template image

Input data examples can be found in `<REPOSITORY>/datasets/`.

In [ ]:
DATA_PATH = pathlib.Path("../datasets/")

In [ ]:
PLOT_NUMBER = 59
MARKER_NUMBER = 1

PLOT_PATH = DATA_PATH / "plot_images" / f"plot{PLOT_NUMBER}.png"
TEMPLATE_PATH = (
    DATA_PATH / "marker_images" / f"plot{PLOT_NUMBER}_marker{MARKER_NUMBER}.png"
)

In [ ]:
source_image = read_image_rgb(PLOT_PATH)
source_template = read_image_rgb(TEMPLATE_PATH)

image = replace_black_pixels(source_image)
template = replace_black_pixels(source_template)

In [ ]:
plt.subplot(1, 2, 1)
draw_image(image)
plt.title("Source image")
plt.xticks([])
plt.yticks([])

plt.subplot(1, 2, 2)
draw_image(template)
plt.title("Chosen marker")
plt.xticks([])
plt.yticks([]);

## Tresholding

In [ ]:
TRESHOLD_VALUE = 190
template_mask = template_tresholding(template, TRESHOLD_VALUE)
additional_template_mask = reconstruct_template_mask(template_mask)

# fig = plt.figure()
# plt.subplot(1, 2, 1)
# draw_image(template_mask)
# plt.xticks([]); plt.yticks([])

# plt.subplot(1, 2, 2)
# draw_image(additional_template_mask)
# plt.xticks([]); plt.yticks([]);

## Template matching

Run template matching algorithm and compute convolutoin (or correlatoin) map.

In [ ]:
correlation_map, _ = template_match(
    image, template, template_mask, norm_result=True
)
correlation_map_additional, _ = template_match(
    image, template, additional_template_mask, norm_result=True
)

## Hough transform

In [ ]:
accumulator = generalized_hough_transform(
    image, template, norm_result=True, crop_result=True
)

assert correlation_map.shape == accumulator.shape

## Combine correlation maps

This step combines correlation map obtained by template matching algorithm and accumulator array from Hough Transform.

In [ ]:
# correlation_map_combined = correlation_map + 0.7 * correlation_map_additional
# correlation_map_combined = normalize_map(correlation_map_combined)

correlation_map_with_hough = correlation_map + 0.6 * accumulator
correlation_map_with_hough = normalize_map(correlation_map_with_hough)

In [ ]:
# imshow_interactive(correlation_map_with_hough)

## Algorithm parameters

The algorithm has 2 parameters:
- Points Number
- Points Density


The impact of these parameters on the final result is shown in the images below:

![](../../docs/images/parameter_search_1.png)

![](../../docs/images/parameter_search_2.png)

In [ ]:
detector = DetectorWidget(
    source_image=source_image,
    template=template,
    correlation_map=correlation_map_with_hough
)

In [ ]:
fig_size = 10
marker_size = 60
marker_color = "yellow"
marker_type = "*"


detector.main_widget(fig_size, marker_size, marker_color, marker_type)

In [ ]:
detected_points_px = detector.get_detections()

# points coordinates in pixels (!)
x = detected_points_px[:, 0]
y = detected_points_px[:, 1]
plt.scatter(x, y)

# Convert coordinates

Map pixel coordinates to factual coordinates

In [ ]:
mapper = CoordinatesMapper(source_image)

In [ ]:
mapper.interactive_widget()

Convert pixel coordinates to factual coordinates

In [ ]:
converter = CoordinatesConverter()
converter.import_parameters_from_mapper(mapper)

In [ ]:
x_px = detected_points_px[:, 0]
y_px = detected_points_px[:, 1]

x_factual, y_factual = converter.from_pixel(x_pixel=x_px, y_pixel=y_px)

In [ ]:
plt.scatter(x_factual, y_factual)

In [ ]:
# dump_coords_csv(
#     x=x_factual,
#     y=y_factual,
#     savepath="tmp.csv"
# )